In [1]:
from sklearn.cluster import KMeans
from transformers import pipeline
import pandas as pd
import numpy as np


c:\Users\ericz\miniconda3\envs\cc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# read data
df = pd.read_csv('comments.csv')

# preprocess
df = df.dropna()
df['comments'] = df['comments'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
comments = df['comments'].tolist()

# load embeddings
embeddings = pd.read_csv('embed.csv')
X = np.array(embeddings)

df[['x', 'y']] = X[:, :2]  # assume first two dimensions

# cluster
cluster = KMeans(n_clusters=10, random_state=42).fit(X)
df['cluster'] = cluster.labels_
df['cluster'] = df['cluster'].astype(str)

c:\Users\ericz\miniconda3\envs\cc\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ericz\miniconda3\envs\cc\lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ericz\miniconda3\envs\cc\lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(
c:\Users\ericz\miniconda3\envs\cc\lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks t

In [7]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

In [10]:
cluster_summary = df.groupby('cluster').agg({'comments': list}).reset_index()
cluster_summary['comments'] = cluster_summary['comments'].apply(lambda x: ' '.join(x)[:1000])
cluster_summary['summary'] = cluster_summary['comments'].apply(lambda x: summarizer(x, min_length=5, max_length=100)[0]['summary_text'])

In [14]:
cluster_summary['summary'][1]

' Mark Ronson and Bruno Mars need a permanent partnership where they keep making funk tracks because believe me we need them  For better or for worse they need them . Bruno Mars is one of the top pop musicians of our age .'